# Model Evaluation and Validation

This notebook provides comprehensive evaluation of trained free testosterone estimation models.

**Contents:**
1. Load trained models and test data
2. Evaluate all models on internal test set
3. Generate Bland-Altman plots for agreement analysis
4. Subgroup analysis by SHBG tertile
5. Bootstrap confidence intervals for key metrics

In [ ]:
import sys
from pathlib import Path

# Add project root to path for imports
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from joblib import load

from freeT.evaluate import (
    evaluate_model,
    bland_altman_stats,
    lins_ccc,
    bootstrap_ci
)
from freeT.train import create_features, stratified_split
from freeT.models import calc_ft_vermeulen

print("Imports successful!")

## 1. Load Data and Models

Load the processed NHANES data and trained models. If real data is unavailable, we generate synthetic data for demonstration.

In [ ]:
# Try to load real NHANES data, fallback to synthetic
data_path = project_root / 'data' / 'processed' / 'nhanes_combined.csv'

if data_path.exists():
    print("Loading real NHANES data...")
    df = pd.read_csv(data_path)
else:
    print("Real data not found. Generating synthetic data for demonstration...")
    np.random.seed(42)
    n_samples = 1000
    
    # Generate physiologically realistic values
    df = pd.DataFrame({
        'seqn': range(1, n_samples + 1),
        'tt_nmoll': np.random.uniform(5, 35, n_samples),
        'shbg_nmoll': np.random.uniform(10, 100, n_samples),
        'alb_gl': np.random.uniform(35, 50, n_samples)
    })
    
    # Calculate reference FT using Vermeulen
    df['ft_reference'] = df.apply(
        lambda row: calc_ft_vermeulen(
            row['tt_nmoll'], row['shbg_nmoll'], row['alb_gl']
        ),
        axis=1
    )
    
    # Add small noise to simulate realistic prediction targets
    df['ft_reference'] = df['ft_reference'] + np.random.normal(0, 0.02, n_samples)
    df['ft_reference'] = df['ft_reference'].clip(lower=0.01)

print(f"Data shape: {df.shape}")
df.head()

In [ ]:
# Create features and split data
X, feature_names = create_features(df)
print(f"Features: {feature_names}")

# Split with stratification by SHBG tertiles
X_train, X_test, y_train, y_test = stratified_split(df, test_size=0.3, random_state=42)

print(f"Train size: {len(y_train)}, Test size: {len(y_test)}")

In [ ]:
# Load trained models if available, otherwise train fresh
models_dir = project_root / 'models'
models = {}

model_files = {
    'Ridge': 'ridge_model.joblib',
    'RandomForest': 'rf_model.joblib',
    'LightGBM': 'lgbm_model.joblib',
    'Best': 'best_model.joblib'
}

for name, filename in model_files.items():
    model_path = models_dir / filename
    if model_path.exists():
        models[name] = load(model_path)
        print(f"Loaded {name} from {filename}")

# If no models found, train them on the fly
if not models:
    print("\nNo saved models found. Training models for demonstration...")
    from freeT.train import train_ridge, train_random_forest, train_lightgbm
    from sklearn.model_selection import train_test_split
    
    # Create validation set for LightGBM
    X_tr, X_val, y_tr, y_val = train_test_split(
        X_train, y_train, test_size=0.2, random_state=42
    )
    
    models['Ridge'] = train_ridge(X_train, y_train)
    print("Trained Ridge")
    
    models['RandomForest'] = train_random_forest(X_train, y_train, n_estimators=100)
    print("Trained RandomForest")
    
    models['LightGBM'] = train_lightgbm(X_tr, y_tr, X_val, y_val)
    print("Trained LightGBM")

print(f"\nModels available: {list(models.keys())}")

## 2. Model Evaluation on Test Set

Evaluate all models using comprehensive metrics: RMSE, MAE, Bias, Pearson r, Lin's CCC, and Bland-Altman statistics.

In [ ]:
# Evaluate all models
results = []

for name, model in models.items():
    if name == 'Best':
        continue  # Skip duplicate best model
    
    y_pred = model.predict(X_test)
    metrics = evaluate_model(y_test, y_pred, name)
    
    results.append({
        'Model': name,
        'RMSE': metrics['rmse'],
        'MAE': metrics['mae'],
        'Bias': metrics['bias'],
        "Pearson r": metrics['r_pearson'],
        "Lin's CCC": metrics['lin_ccc'],
        'LOA Lower': metrics['ba_stats']['loa_lower'],
        'LOA Upper': metrics['ba_stats']['loa_upper']
    })

# Also evaluate Vermeulen baseline (mechanistic solver)
ft_vermeulen = X_test[:, feature_names.index('ft_vermeulen')]
vermeulen_metrics = evaluate_model(y_test, ft_vermeulen, 'Vermeulen')

results.append({
    'Model': 'Vermeulen',
    'RMSE': vermeulen_metrics['rmse'],
    'MAE': vermeulen_metrics['mae'],
    'Bias': vermeulen_metrics['bias'],
    "Pearson r": vermeulen_metrics['r_pearson'],
    "Lin's CCC": vermeulen_metrics['lin_ccc'],
    'LOA Lower': vermeulen_metrics['ba_stats']['loa_lower'],
    'LOA Upper': vermeulen_metrics['ba_stats']['loa_upper']
})

# Create comparison table
results_df = pd.DataFrame(results)
results_df = results_df.round(4)
print("\n=== Model Comparison ===")
results_df

## 3. Bland-Altman Plots

Bland-Altman plots show the agreement between predicted and true values, with mean bias and 95% limits of agreement (LOA).

In [ ]:
def plot_bland_altman(y_true, y_pred, model_name, ax):
    """Generate Bland-Altman plot on given axes."""
    stats = bland_altman_stats(y_true, y_pred)
    
    mean_values = (y_true + y_pred) / 2
    differences = y_pred - y_true
    
    ax.scatter(mean_values, differences, alpha=0.5, s=20)
    ax.axhline(y=stats['mean_bias'], color='red', linestyle='-', 
               label=f"Mean bias: {stats['mean_bias']:.4f}")
    ax.axhline(y=stats['loa_upper'], color='gray', linestyle='--',
               label=f"Upper LOA: {stats['loa_upper']:.4f}")
    ax.axhline(y=stats['loa_lower'], color='gray', linestyle='--',
               label=f"Lower LOA: {stats['loa_lower']:.4f}")
    ax.axhline(y=0, color='black', linestyle=':', alpha=0.5)
    
    ax.set_xlabel('Mean of True and Predicted (nmol/L)')
    ax.set_ylabel('Difference (Predicted - True)')
    ax.set_title(f'{model_name}')
    ax.legend(fontsize=8, loc='best')
    ax.grid(True, alpha=0.3)

# Create Bland-Altman plots for all models
n_models = len([m for m in models.keys() if m != 'Best']) + 1  # +1 for Vermeulen
fig, axes = plt.subplots(1, n_models, figsize=(5 * n_models, 4))

if n_models == 1:
    axes = [axes]

plot_idx = 0
for name, model in models.items():
    if name == 'Best':
        continue
    y_pred = model.predict(X_test)
    plot_bland_altman(y_test, y_pred, name, axes[plot_idx])
    plot_idx += 1

# Add Vermeulen baseline
plot_bland_altman(y_test, ft_vermeulen, 'Vermeulen', axes[plot_idx])

plt.suptitle('Bland-Altman Plots: Model Agreement Analysis', y=1.02, fontsize=14)
plt.tight_layout()
plt.savefig(project_root / 'notebooks' / 'bland_altman_plots.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. Predicted vs Actual Plots

In [ ]:
# Create predicted vs actual plots
fig, axes = plt.subplots(1, n_models, figsize=(5 * n_models, 4))

if n_models == 1:
    axes = [axes]

plot_idx = 0
for name, model in models.items():
    if name == 'Best':
        continue
    y_pred = model.predict(X_test)
    ccc = lins_ccc(y_test, y_pred)
    
    axes[plot_idx].scatter(y_test, y_pred, alpha=0.5, s=20)
    
    # Add identity line
    min_val = min(y_test.min(), y_pred.min())
    max_val = max(y_test.max(), y_pred.max())
    axes[plot_idx].plot([min_val, max_val], [min_val, max_val], 'r--', label='Identity')
    
    axes[plot_idx].set_xlabel('True FT (nmol/L)')
    axes[plot_idx].set_ylabel('Predicted FT (nmol/L)')
    axes[plot_idx].set_title(f"{name}\nCCC = {ccc:.4f}")
    axes[plot_idx].legend()
    axes[plot_idx].grid(True, alpha=0.3)
    plot_idx += 1

# Add Vermeulen baseline
ccc = lins_ccc(y_test, ft_vermeulen)
axes[plot_idx].scatter(y_test, ft_vermeulen, alpha=0.5, s=20)
min_val = min(y_test.min(), ft_vermeulen.min())
max_val = max(y_test.max(), ft_vermeulen.max())
axes[plot_idx].plot([min_val, max_val], [min_val, max_val], 'r--', label='Identity')
axes[plot_idx].set_xlabel('True FT (nmol/L)')
axes[plot_idx].set_ylabel('Predicted FT (nmol/L)')
axes[plot_idx].set_title(f"Vermeulen\nCCC = {ccc:.4f}")
axes[plot_idx].legend()
axes[plot_idx].grid(True, alpha=0.3)

plt.suptitle('Predicted vs Actual Free Testosterone', y=1.02, fontsize=14)
plt.tight_layout()
plt.savefig(project_root / 'notebooks' / 'predicted_vs_actual.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. Subgroup Analysis by SHBG Tertile

Analyze model performance across SHBG subgroups (Low, Medium, High) to identify potential systematic biases.

In [ ]:
# Create SHBG tertiles for test set
shbg_test = X_test[:, feature_names.index('shbg_nmoll')]
tertile_labels = pd.qcut(shbg_test, q=3, labels=['Low', 'Medium', 'High'], duplicates='drop')

print("SHBG tertile distribution in test set:")
print(pd.Series(tertile_labels).value_counts().sort_index())

# Subgroup analysis for each model
subgroup_results = []

for tertile in ['Low', 'Medium', 'High']:
    mask = tertile_labels == tertile
    y_test_sub = y_test[mask]
    X_test_sub = X_test[mask]
    
    for name, model in models.items():
        if name == 'Best':
            continue
        
        y_pred_sub = model.predict(X_test_sub)
        metrics = evaluate_model(y_test_sub, y_pred_sub, name)
        
        subgroup_results.append({
            'SHBG Tertile': tertile,
            'Model': name,
            'RMSE': metrics['rmse'],
            'Bias': metrics['bias'],
            "Lin's CCC": metrics['lin_ccc'],
            'N': len(y_test_sub)
        })
    
    # Vermeulen baseline
    ft_verm_sub = X_test_sub[:, feature_names.index('ft_vermeulen')]
    metrics = evaluate_model(y_test_sub, ft_verm_sub, 'Vermeulen')
    subgroup_results.append({
        'SHBG Tertile': tertile,
        'Model': 'Vermeulen',
        'RMSE': metrics['rmse'],
        'Bias': metrics['bias'],
        "Lin's CCC": metrics['lin_ccc'],
        'N': len(y_test_sub)
    })

subgroup_df = pd.DataFrame(subgroup_results)
print("\n=== Subgroup Analysis by SHBG Tertile ===")
subgroup_df.round(4)

In [ ]:
# Visualize subgroup performance
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

metrics_to_plot = ['RMSE', 'Bias', "Lin's CCC"]
tertiles = ['Low', 'Medium', 'High']
model_names = [m for m in models.keys() if m != 'Best'] + ['Vermeulen']

for idx, metric in enumerate(metrics_to_plot):
    ax = axes[idx]
    
    x = np.arange(len(tertiles))
    width = 0.2
    
    for i, model in enumerate(model_names):
        values = [subgroup_df[(subgroup_df['Model'] == model) & 
                             (subgroup_df['SHBG Tertile'] == t)][metric].values[0] 
                  for t in tertiles]
        ax.bar(x + i * width, values, width, label=model)
    
    ax.set_xlabel('SHBG Tertile')
    ax.set_ylabel(metric)
    ax.set_title(f'{metric} by SHBG Tertile')
    ax.set_xticks(x + width * (len(model_names) - 1) / 2)
    ax.set_xticklabels(tertiles)
    ax.legend(fontsize=8)
    ax.grid(True, alpha=0.3, axis='y')

plt.suptitle('Model Performance Across SHBG Subgroups', y=1.02, fontsize=14)
plt.tight_layout()
plt.savefig(project_root / 'notebooks' / 'subgroup_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Bootstrap Confidence Intervals

Calculate 95% confidence intervals for key metrics using bootstrap resampling.

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

def rmse_metric(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def mae_metric(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)

def bias_metric(y_true, y_pred):
    return np.mean(y_pred - y_true)

# Calculate bootstrap CIs for best performing model
best_model_name = results_df.loc[results_df['RMSE'].idxmin(), 'Model']
print(f"Calculating bootstrap CIs for best model: {best_model_name}")

if best_model_name == 'Vermeulen':
    y_pred_best = ft_vermeulen
else:
    y_pred_best = models[best_model_name].predict(X_test)

ci_results = []

for metric_name, metric_func in [('RMSE', rmse_metric), 
                                   ('MAE', mae_metric),
                                   ('Bias', bias_metric),
                                   ("Lin's CCC", lins_ccc)]:
    print(f"  Computing CI for {metric_name}...")
    lower, upper, mean = bootstrap_ci(
        y_test, y_pred_best, metric_func, 
        n_bootstrap=2000, random_state=42
    )
    ci_results.append({
        'Metric': metric_name,
        'Point Estimate': mean,
        '95% CI Lower': lower,
        '95% CI Upper': upper
    })

ci_df = pd.DataFrame(ci_results).round(4)
print(f"\n=== Bootstrap 95% CI for {best_model_name} ===")
ci_df

## 7. Summary and Conclusions

### Key Findings

This evaluation notebook demonstrates:

1. **Model Comparison**: All models were evaluated on the test set using multiple metrics
2. **Agreement Analysis**: Bland-Altman plots show the bias and limits of agreement
3. **Subgroup Performance**: Models may perform differently across SHBG tertiles
4. **Uncertainty Quantification**: Bootstrap CIs provide confidence bounds for metrics

### Target Metrics (from PRD)
- Mean bias < ±0.5 nmol/L
- Lin's CCC ≥ 0.90

In [ ]:
# Final summary
print("=" * 60)
print("FINAL MODEL EVALUATION SUMMARY")
print("=" * 60)

print(f"\nBest Model: {best_model_name}")
best_metrics = results_df[results_df['Model'] == best_model_name].iloc[0]

print(f"\nPerformance Metrics:")
print(f"  RMSE:      {best_metrics['RMSE']:.4f} nmol/L")
print(f"  MAE:       {best_metrics['MAE']:.4f} nmol/L")
print(f"  Bias:      {best_metrics['Bias']:.4f} nmol/L")
print(f"  Pearson r: {best_metrics['Pearson r']:.4f}")
print(f"  Lin's CCC: {best_metrics["Lin's CCC"]:.4f}")

print(f"\nTarget Validation:")
bias_ok = abs(best_metrics['Bias']) < 0.5
ccc_ok = best_metrics["Lin's CCC"] >= 0.90

print(f"  Mean bias < ±0.5 nmol/L: {'✓ PASS' if bias_ok else '✗ FAIL'}")
print(f"  Lin's CCC ≥ 0.90:        {'✓ PASS' if ccc_ok else '✗ FAIL'}")

print("\n" + "=" * 60)
print("Evaluation complete!")